# Part 1 -- Setup & Collection 

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [7]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter


In [9]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

AttributeError: 'module' object has no attribute 'STOPWORDS'

In [ ]:
!pip install pymongo pyquery tweepy
import pymongo
import tweepy
# from tqdm import tqdm 

In [ ]:
# Replace the API_KEY and API_SECRET with your application's key and secret
auth = tweepy.AppAuthHandler(my_keys['CONSUMER_KEY'], my_keys['CONSUMER_SECRET'])

# Authorize twitter, initialize tweepy
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [7]:
#identify port for security
cli = pymongo.MongoClient(host='52.27.11.214', port=27016)

In [8]:
# Instantiates when you put data in 
task_collection = cli.twitter_db.task_collection
completed_collection = cli.twitter_db.completed_collection
# tweets_collection = cli.twitter_db.tweets_collection
cli.twitter_db.collection_names()

[u'task_collection']

In [9]:
print(task_collection.count())
print(completed_collection.count())

30
0


In [56]:
task_collection.drop()
completed_collection.drop()

In [10]:
chdir('//home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/tools/GetOldTweets-python-master')
!pip install -r requirements.txt
import got

    100% |████████████████████████████████| 3.8MB 147kB/s eta 0:00:01
  Running setup.py bdist_wheel for lxml ... - \ | error
  Complete output from command /opt/conda/envs/python2/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-E4xHw2/lxml/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpRKoEDqpip-wheel- --python-tag cp27:
  Building lxml version 3.5.0.
  Building without Cython.
  ERROR: /bin/sh: 1: xslt-config: not found
  
  ** make sure the development packages of libxml2 and libxslt are installed **
  
  Using build configuration of libxslt
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-2.7
  creating build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/__init__.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/usedoctest.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lx

### Implement distributed processing from scratch - run this on multiple AWS instances:

In [57]:
twitter_30_df = pd.read_csv('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/data/twitter_users_30.csv')
handle = list(twitter_30_df['Handle'])

In [58]:
lookup_list = []
for h in handle:
    lookup_dict = {'handle': h,
                   'status': 'pending'}
    lookup_list.append(lookup_dict)

In [59]:
lookup_list

[{'handle': 'BarackObama', 'status': 'pending'},
 {'handle': 'cnnbrk', 'status': 'pending'},
 {'handle': 'BillGates', 'status': 'pending'},
 {'handle': 'realDonaldTrump', 'status': 'pending'},
 {'handle': 'TheEconomist', 'status': 'pending'},
 {'handle': 'aplusk', 'status': 'pending'},
 {'handle': 'HillaryClinton', 'status': 'pending'},
 {'handle': 'TechCrunch', 'status': 'pending'},
 {'handle': 'elonmusk', 'status': 'pending'},
 {'handle': 'NewYorker', 'status': 'pending'},
 {'handle': 'mcuban', 'status': 'pending'},
 {'handle': 'jack', 'status': 'pending'},
 {'handle': 'MarketWatch', 'status': 'pending'},
 {'handle': 'CNBC', 'status': 'pending'},
 {'handle': 'ForbesTech', 'status': 'pending'},
 {'handle': 'sacca', 'status': 'pending'},
 {'handle': 'SAI', 'status': 'pending'},
 {'handle': 'paulg', 'status': 'pending'},
 {'handle': 'themotleyfool', 'status': 'pending'},
 {'handle': 'ReformedBroker', 'status': 'pending'},
 {'handle': 'StockTwits', 'status': 'pending'},
 {'handle': 'cnnt

In [60]:
for dictionary in lookup_list:
    task_collection.insert_one(dictionary)

In [61]:
task_collection.count()

30

In [11]:
def get_tweets_to_mongo(lookup):

    maxTweets = 10000000 # Some arbitrary large number
    tweetsPerQry = 200  # this is the max the API permits

    # If results from a specific ID onwards are reqd, set since_id to that ID.
    # else default to no lower limit, go as far back as API allows
    sinceId = None

    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1E10

    tweetCount = 0
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = lookup, count=tweetsPerQry)
                else:
                    new_tweets = api.user_timeline(screen_name = lookup, count=tweetsPerQry,
                                                    since_id=sinceId)
            else:

                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = lookup, count=tweetsPerQry,
                                                    max_id=str(max_id - 1))
                else:
                    new_tweets = api.user_timeline(screen_name = lookup, count=tweetsPerQry,
                                                    max_id=str(max_id - 1),
                                                    since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                tweet_dict = {'username': tweet.user.screen_name,
                              'timestamp': tweet.created_at, 
                              'text': tweet.text.encode("utf-8")}
                mongo_response = completed_collection.insert_one(tweet_dict)

            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets for user: {1}".format(tweetCount, lookup))
            max_id = new_tweets[-1].id  

        except tweepy.TweepError as e:
            # Just exit if any error
            task_collection.insert_one({'handle': lookup, 'status': 'failed'})
            print("some error : " + str(e))
            break

    print ("Downloaded {0} tweets for user: {1} & saved to Mongo\n".format(tweetCount, lookup))

In [12]:
from datetime import datetime
start = datetime.now()

while True:
    # Get a user from the mongo collection
    task = task_collection.find_one_and_delete({'status': 'pending'})

    if type(task) != dict:
        print('Done!')
        break

    # Assign the handle to task_handle
    task_handle = task['handle']
        
        
    get_tweets_to_mongo(task_handle)
    task_collection.insert_one({'handle': task['handle'], 'status': 'complete'})
    
end = datetime.now()
print(end - start)

Downloaded 200 tweets for user: BarackObama
Downloaded 400 tweets for user: BarackObama
Downloaded 600 tweets for user: BarackObama
Downloaded 800 tweets for user: BarackObama
Downloaded 1000 tweets for user: BarackObama
Downloaded 1200 tweets for user: BarackObama
Downloaded 1400 tweets for user: BarackObama
Downloaded 1600 tweets for user: BarackObama
Downloaded 1800 tweets for user: BarackObama
Downloaded 2000 tweets for user: BarackObama
Downloaded 2200 tweets for user: BarackObama
Downloaded 2399 tweets for user: BarackObama
Downloaded 2599 tweets for user: BarackObama
Downloaded 2796 tweets for user: BarackObama
Downloaded 2996 tweets for user: BarackObama
Downloaded 3196 tweets for user: BarackObama
Downloaded 3234 tweets for user: BarackObama
No more tweets found
Downloaded 3234 tweets for user: BarackObama & saved to Mongo

Downloaded 200 tweets for user: BillGates
Downloaded 400 tweets for user: BillGates
Downloaded 600 tweets for user: BillGates
Downloaded 800 tweets for use

In [64]:
completed_collection.count()

87258

In [65]:
87258.0/90000.0
#96.95% of possible tweets achieved - in 04:26 minutes

0.9695333333333334

In [52]:
87244.0/90000.0
#96.93% of possible tweets achieved

0.9693777777777778

In [23]:
87222.0/90000.0
#96.91% of possible tweets achieved

0.9691333333333333

In [54]:
curs = task_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

,_id,handle,status
0,59358a7f567536001f22525e,BarackObama,complete
1,59358a8a567536001f225f07,cnnbrk,complete
2,59358a92567536001f226852,BillGates,complete
3,59358a9b567536001f227504,realDonaldTrump,complete
4,59358aa5567536001f22818b,TheEconomist,complete
5,59358aaf567536001f228e0b,aplusk,complete
6,59358ab9567536001f229aac,HillaryClinton,complete
7,59358ac3567536001f22a736,TechCrunch,complete
8,59358acc567536001f22b2f0,elonmusk,complete
9,59358ad6567536001f22bf7f,NewYorker,complete


In [22]:
curs = tweets_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

,_id,text,timestamp,username
0,5931f76f307e010746f635f4,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama
1,5931f76f307e010746f635f5,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama
2,5931f76f307e010746f635f6,Good to see my friend Prince Harry in London t...,2017-05-27 13:15:25,BarackObama
3,5931f76f307e010746f635f7,"Through faith, love, and resolve the character...",2017-05-25 14:13:35,BarackObama
4,5931f76f307e010746f635f8,Our hearts go out to those killed and wounded ...,2017-05-23 16:56:14,BarackObama
5,5931f76f307e010746f635f9,"Excited to hear from Sierra, Imani, Filiz, and...",2017-05-22 21:16:23,BarackObama
6,5931f76f307e010746f635fa,Happy Mother's Day to my love and partner on t...,2017-05-14 14:01:16,BarackObama
7,5931f76f307e010746f635fb,"We're rolling up our sleeves again, back where...",2017-05-03 19:42:16,BarackObama
8,5931f76f307e010746f635fc,"Well said, Jimmy. That's exactly why we fought...",2017-05-02 17:19:26,BarackObama
9,5931f76f307e010746f635fd,RT @ObamaFoundation: It’s a beautiful day on t...,2017-04-24 19:48:17,BarackObama


In [ ]:
# def get_tweets_to_mongo(lookup):

#     maxTweets = 10000000 # Some arbitrary large number
#     tweetsPerQry = 200  # this is the max the API permits

#     for l in lookup:
#         # If results from a specific ID onwards are reqd, set since_id to that ID.
#         # else default to no lower limit, go as far back as API allows
#         sinceId = None

#         # If results only below a specific ID are, set max_id to that ID.
#         # else default to no upper limit, start from the most recent tweet matching the search query.
#         max_id = -1E10

#         tweetCount = 0
#         while tweetCount < maxTweets:
#             try:
#                 if (max_id <= 0):
#                     if (not sinceId):
#                         new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry)
#                     else:
#                         new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
#                                                     since_id=sinceId)
#                 else:

#                     if (not sinceId):
#                         new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
#                                                     max_id=str(max_id - 1))
#                     else:
#                         new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
#                                                     max_id=str(max_id - 1),
#                                                     since_id=sinceId)
#                 if not new_tweets:
#                     print("No more tweets found")
#                     break
#                 for tweet in new_tweets:
#                     tweet_dict = {'username': tweet.user.screen_name,
#                                   'timestamp': tweet.created_at, 
#                                   'text': tweet.text.encode("utf-8")}
#         #             writer = csv.writer(f)
#         #             writer.writerow(["id","created_at","text"])
#         #             writer.writerows(outtweets)
#                     mongo_response = tweets_collection.insert_one(tweet_dict)

#                 tweetCount += len(new_tweets)
#                 print("Downloaded {0} tweets for user: {1}".format(tweetCount, l))
#                 max_id = new_tweets[-1].id  

#             except tweepy.TweepError as e:
#                 # Just exit if any error
#                 print("some error : " + str(e))
#                 break

#         print ("Downloaded {0} tweets for user: {1} & saved to Mongo\n".format(tweetCount, l))

In [ ]:
from datetime import datetime
start = datetime.now()
get_tweets_to_mongo(handle)
end = datetime.now()
print(end - start)

In [ ]:
tweets_collection.count()

In [ ]:
curs = tweets_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

In [ ]:
df.shape